## Montamos el drive de google ya que estamos trabajando en Colab

In [18]:
# Como estamos trabajando en colab y tengo los archivos en el drive importamos la libreria y montamos
from google.colab import drive #para utilizar google drive como fuente de archivos
drive.mount('/content/drive') #montamos la carpeta del drivec

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importamos Librerias necesarias para la lectura del archivo y visualización de datos

In [19]:
import json
import polars as pl

### Definimos una funcion para la lectura de archivos json

In [ ]:
def leer_json (n):
    data_list = []

    with open(f'/content/drive/MyDrive/PF Google & Yelp/{n}.json', 'r') as file:
        for line in file:
            # Cargar cada objeto JSON por separado
            data = json.loads(line)
            data_list.append(data)

    # Ahora data_list es una lista que contiene todos los objetos JSON del archivo
    df = pl.DataFrame(data_list)
    return df

### Definimos una funcion para las transformaciones

In [ ]:
def transform (dft):
    dft = dft.drop(['address', 'description', 'price', 'hours', 'MISC', 'state', 'relative_results'])
    dft = dft.explode('category')
    dft = dft.filter(dft['category'].str.contains(f'(?i)hotel'))
    dft = dft.unique(subset=['gmap_id'], keep='first')
    return dft

### Iniciamos un bucle para la lectura de los 11 archivos

In [ ]:
for i in range(1,12):
    df1 = leer_json(i)
    df1 = transform(df1)
    df1.write_parquet(f'/content/drive/MyDrive/PF Google & Yelp/r{i}.parquet')

### Instalamos la dependencia para la API de google maps

In [20]:
!pip install -U googlemaps

### Importamos la libreria para utilizar la API de google maps

In [21]:
import googlemaps

### Obtenemos la API Key del archivo y la definimos

In [22]:
# Configurar la API Key de Google Maps
with open('/content/drive/MyDrive/PF Google & Yelp/APK.txt', 'r') as file:
    api_key = file.read().strip()

gmaps = googlemaps.Client(key=api_key)

### Definimos una función para obtener el Condado, Ciudad, Estado y Pais

In [26]:
# Definir la función para obtener city y country a partir de coordenadas
def obtener_city_country(latitudes, longitudes):
    counties, cities, states, countries, = [], [], [], []

    for lat, lon in zip(latitudes, longitudes):
        resultado = gmaps.reverse_geocode((lat, lon))
        county, city, state, country = None, None, None, None

        if resultado:
            for component in resultado[0]['address_components']:
                if 'locality' in component['types'] and not city:
                    city = component['long_name']

                elif 'administrative_area_level_2' in component['types'] and not county:
                    county = component['long_name']

                elif 'administrative_area_level_1' in component['types'] and not state:
                    state = component['long_name']

                elif 'country' in component['types'] and not country:
                    country = component['long_name']
                elif city and county and state and country:
                  break

        counties.append(county)
        cities.append(city)
        states.append(state)
        countries.append(country)

    return pl.Series(counties), pl.Series(cities), pl.Series(states), pl.Series(countries)

### Iniciamos un bucle para leer los archivos limpios y obtener los datos de la localidad

In [27]:
for i in range(1,12):
    df2 = pl.read_parquet(f'/content/drive/MyDrive/PF Google & Yelp/r{i}.parquet')
    counties, cities, states, countries = obtener_city_country(df2['latitude'], df2['longitude'])
    df2 = df2.with_columns(
    County=counties,
    City=cities,
    State=states,
    Country=countries)
    df2.write_parquet(f'/content/drive/MyDrive/PF Google & Yelp/ru{i}.parquet')

### Leemos el primer archivo con los datos de la localidad y filtramos por las ciudades de interes, luego continuamos con los demas archivos y anidamos los dataframes

In [33]:
dfa = pl.read_parquet('/content/drive/MyDrive/PF Google & Yelp/ru1.parquet')
dfa = dfa.filter((dfa['City'].str.contains(f'(?i)miami')) | (dfa['City'].str.contains(f'(?i)new york') | (dfa['City'].str.contains(f'(?i)orlando'))))
for i in range(2,12):
    dfb = pl.read_parquet(f'/content/drive/MyDrive/PF Google & Yelp/ru{i}.parquet')
    dfb = dfb.filter((dfb['City'].str.contains(f'(?i)miami')) | (dfb['City'].str.contains(f'(?i)new york') | (dfb['City'].str.contains(f'(?i)orlando'))))
    dfa.extend(dfb)

### Guardamos el Dataset resultante en un archivo csv

In [35]:
dfa.write_csv('/content/drive/MyDrive/PF Google & Yelp/hoteles.csv', separator=",")